In [1]:
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

In [19]:
x = 1
x>=1

True

In [2]:
df = pd.read_csv('./Binance_BTCUSDT_minute.csv', parse_dates=["date"])
df = df.sort_values(by='date').reset_index(drop=True)
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284


In [3]:
df["prev_close"] = df.shift(1)['close']
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,prev_close
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342,NaN
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256,10316.00
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193,10319.50
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187,10319.34
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284,10319.00


In [4]:
tqdm.pandas(desc="Example Desc")
df['close_change'] = df.progress_apply(
    lambda row: 0 if np.isnan(row.prev_close) else row.close - row.prev_close,
    axis=1
)
df.head()

Example Desc: 100%|██████████| 273288/273288 [00:13<00:00, 19892.11it/s]


,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,prev_close,close_change
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342,NaN,0.00
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256,10316.00,3.50
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193,10319.50,-0.16
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187,10319.34,-0.34
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284,10319.00,0.47


In [5]:
rows = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    row_data = dict(
        day_of_week = row.date.dayofweek,
        day_of_month = row.date.day,
        week_of_year = row.date.week,
        month = row.date.month,
        open = row.open,
        high = row.high,
        low = row.low,
        close_change = row.close_change,
        close= row.close
        )
    rows.append(row_data)
feature_df = pd.DataFrame(rows)
feature_df.head()

100%|██████████| 273288/273288 [00:49<00:00, 5511.27it/s]


,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,4,11,37,9,10317.00,10317.01,10314.36,0.00,10316.00
1,4,11,37,9,10316.00,10321.09,10316.00,3.50,10319.50
2,4,11,37,9,10319.50,10320.31,10318.49,-0.16,10319.34
3,4,11,37,9,10319.34,10320.14,10318.89,-0.34,10319.00
4,4,11,37,9,10318.99,10319.72,10316.58,0.47,10319.47


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(feature_df)
total_df = pd.DataFrame(
    scaler.transform(feature_df),
    index = feature_df.index,
    columns = feature_df.columns
)
total_df.head()

,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,0.666667,0.333333,0.692308,0.727273,0.003239,0.003048,0.003440,0.621447,0.003219
1,0.666667,0.333333,0.692308,0.727273,0.003220,0.003127,0.003472,0.622469,0.003287
2,0.666667,0.333333,0.692308,0.727273,0.003288,0.003112,0.003520,0.621401,0.003284
3,0.666667,0.333333,0.692308,0.727273,0.003285,0.003108,0.003528,0.621348,0.003277
4,0.666667,0.333333,0.692308,0.727273,0.003278,0.003100,0.003483,0.621585,0.003286


In [8]:
class model_s(nn.Module):
    def __init__(self, n_features, n_hidden=128, n_layers=2):
        super(model_s,self).__init__()
        self.n_hidden = n_hidden

        self.lstm = nn.LSTM(
            input_size = n_features,
            hidden_size = n_hidden,
            batch_first = True,
            num_layers = n_layers,
            dropout = 0.2
        )

        self.regressor = nn.Linear(n_hidden, 1)
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden_, _) = self.lstm(x)
        out = hidden_[-1]
        fin = self.regressor(out)

        return fin


In [9]:
model = model_s(9)

In [10]:
model(torch.rand(1,1,9))

tensor([[-0.0022]], grad_fn=<AddmmBackward>)